In [ ]:
%pip install langchain openai cohere tiktoken

In [12]:
import json
import random

example_file = open("../json/example.json","r")
test_file = open("../json/test.json","r")
format = open("../json/format.json","r").read()
tools = open("../json/tools.json","r").read()


In [13]:
# get examples
test_examples = json.loads(test_file.read())
examples = json.loads(example_file.read())

In [15]:
prompt_template= """
{tools_template}

{format_template}

{example_template}

{start_template}
"""

tools_template = """
Use the following tools and answer the given question using given answer format, 
{tools}
"""
answer_format_template = """
Generated answer for any given question should have following format
{format}
"""

example_template = """
Following are the example of question-answer pairs for the abovementioned tools
{example1}
"""

start_template = """
To reference the value of the ith tool in the chain, use $$PREV[i] as argument value. i = 0, 1, .. j-1; j = current tool's index in the array
If the query could not be answered with the given set of tools, output an empty list instead.

Empty list should look like []. Whenever you want to access information about current user to pass it further, you need to make a call to whoami api tool.

Answer the following question based on instructions provided above.
question: {question}
answer:{{
"question":
"answer":
}}"""

In [16]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

full_prompt = PromptTemplate.from_template(prompt_template)
tools_prompt = PromptTemplate.from_template(tools_template)
answer_format_prompt = PromptTemplate.from_template(answer_format_template)
example_prompt = PromptTemplate.from_template(example_template)
start_prompt = PromptTemplate.from_template(start_template)

input_prompt = [
    ("tools_template",tools_prompt),
    ("format_template",answer_format_prompt),
    ("example_template",example_prompt),
    ("start_template",start_prompt)
]
prompt = PipelinePromptTemplate(final_prompt=full_prompt,pipeline_prompts=input_prompt)
prompt.input_variables

['example1', 'tools', 'format', 'question']

In [ ]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
config = open("config.txt","r").read()
OPENAI_API_KEY=config
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

chain = LLMChain(llm = ChatOpenAI(model='gpt-4'), prompt=prompt)

In [23]:
response = []
for test in test_examples:

    print(test['question_id'])
    # input = prompt.format(question=test['question'],tools=tools,format=format,examples=examples)
    output = chain.invoke({'question':test['question'],'tools':tools,'format':format,'example1':examples[0]})

    answer = json.loads(output['text'])
    if (answer == []):
        answer = {'question':test['question'],'answer':[]}
    answer['question_id'] = test['question_id']
    response.append(answer)


2
3
6
7
8


In [24]:
outfile = open("../output/zero shot with one example/GPT_4_res.json","w")
json.dump(response,outfile,indent=4)
outfile.close()